In [22]:
import plotly
plotly.__version__
import pandas as pd

In [23]:
mdata = pd.read_csv("DataSets/belly_button_metadata.csv")
mdata.head()

,sample,EVENT,ETHNICITY,GENDER,AGE,WFREQ,BBTYPE,LOCATION,COUNTRY012,ZIP012,...,DOG,CAT,IMPSURFACE013,NPP013,MMAXTEMP013,PFC013,IMPSURFACE1319,NPP1319,MMAXTEMP1319,PFC1319
0,940,BellyButtonsScienceOnline,Caucasian,F,24.0,2.0,I,Beaufort/NC,usa,22306,...,no,no,8852.0,37.172222,54.5,NaN,1,NaN,33.990002,25.5
1,941,NaN,Caucasian/Midleastern,F,34.0,1.0,I,Chicago/IL,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,943,BellyButtonsScienceOnline,Caucasian,F,49.0,1.0,I,Omaha/NE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,944,BellyButtonsScienceOnline,European,M,44.0,1.0,I,NewHaven/CT,usa,7079,...,no,yes,NaN,35.816666,16.0,NaN,0,6567.0,32.403332,28.5
4,945,BellyButtonsScienceOnline,Caucasian,F,48.0,1.0,I,Philidelphia/PA,usa,84404,...,no,no,NaN,37.783333,4.0,NaN,0,5613.0,33.634445,24.0


In [24]:
bbdata = pd.read_csv("Datasets/belly_button_data.csv")
bbdata.head()

,otu_id,otu_label,940,941,943,944,945,946,947,948,...,1562,1563,1564,1572,1573,1574,1576,1577,1581,1601
0,1,Archaea;Euryarchaeota;Halobacteria;Halobacteri...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2,Archaea;Euryarchaeota;Halobacteria;Halobacteri...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,3,Archaea;Euryarchaeota;Halobacteria;Halobacteri...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,4,Archaea;Euryarchaeota;Methanobacteria;Methanob...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,5,Archaea;Euryarchaeota;Methanobacteria;Methanob...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [172]:
import os

import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)

In [173]:
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///StarterCode/Belly_Button_Biodiversity/db/belly_button_biodiversity.sqlite"
db = SQLAlchemy(app)

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(db.engine, reflect=True)

/Users/mgarza/anaconda3/lib/python3.6/site-packages/flask_sqlalchemy/__init__.py:794: FSADeprecationWarning:

SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.



In [174]:
Base.classes.keys()

['otu', 'samples', 'samples_metadata']

In [175]:
Samples_Metadata = Base.classes.samples_metadata
Samples = Base.classes.samples
Otu = Base.classes.otu

In [176]:
session = Session(engine)

In [194]:
def idCollector (sample_id):
    id = [sample_id]
    results = session.query(Samples).options(load_only(sample_id)).order_by(desc(sample_id))
    
    collector = []
    names = []
    values = []
    
    for result in results[:10]:
        row = result.__dict__    
        names.append(f"Otu - {row['otu_id']}")
        values.append(row[f"{sample_id}"])
    
    values = [round((x/sum(values))*100,2) for x in values]
        
    return {
        'Name':names,
        'values':values
    }
# function is functioning

In [198]:
def infoCollector(sample_id):
    info_id = sample_id[3:]
    results = session.query(Samples_Metadata).filter(Samples_Metadata.SAMPLEID == info_id).first()
    
    print(results)
    
    return {
        'age':results.AGE,
        'gender':results.GENDER,
        'ethnicity':results.ETHNICITY,
        'location':results.LOCATION,
        'source':results.EVENT
    }
# function is functioning

In [200]:
def OtuCollector():
    results = session.query(Samples).all()
    x = []
    y = []
    
    for result in results:
        row = result.__dict__
        otu_values = list(row.values())
        x.append(result.otu_id)
        y.append(sum(otu_values[1:]))
        
    return {
        'x':x,
        'y':y
    }
# function is functioning      

In [202]:
def idList():
    columns = Samples.__table__.columns.keys()
    return columns[1:]
# function is functioning